# Train Yolov7

In [ ]:
#!python train.py --freeze 29 --batch-size 32 --data "../data/FineTuned_YOLO/data.yaml" --img 1080 1920 --cfg cfg/training/yolov7-e6e.yaml --weight "./yolov7-e6e.pt" --name yolov7-custom-freeze --epoch 200
#!python detect.py --weights "./runs/train/yolov7-custom-freeze/weights/best.pt" --conf 0.25 --source ./video1_test.mp4

In [ ]:
# https://learnopencv.com/fine-tuning-yolov7-on-custom-dataset/

# Test Model

In [6]:
import sys
sys.path.append('./yolov7')
import cv2
from pathlib import Path
import torch
import numpy as np
# Verify inference
import numpy as np
from PIL import Image


from yolov7.models.yolo import Model
from yolov7.utils.general import check_requirements, set_logging
from yolov7.utils.google_utils import attempt_download
from yolov7.utils.torch_utils import select_device

### Load model

In [5]:
## Load model
dependencies = ['torch', 'yaml']
check_requirements(Path("./yolov7").parent / 'requirements.txt', exclude=('pycocotools', 'thop'))
set_logging()

def custom(path_or_model='path/to/model.pt', autoshape=True):
    """custom mode

    Arguments (3 options):
        path_or_model (str): 'path/to/model.pt'
        path_or_model (dict): torch.load('path/to/model.pt')
        path_or_model (nn.Module): torch.load('path/to/model.pt')['model']

    Returns:
        pytorch model
    """
    model = torch.load(path_or_model, map_location=torch.device('cpu')) if isinstance(path_or_model, str) else path_or_model  # load checkpoint

    if isinstance(model, dict):
        model = model['ema' if model.get('ema') else 'model']  # load model

    hub_model = Model(model.yaml).to(next(model.parameters()).device)  # create
    hub_model.load_state_dict(model.float().state_dict())  # load state_dict
    hub_model.names = model.names  # class names
    if autoshape:
        hub_model = hub_model.autoshape()  # for file/URI/PIL/cv2/np inputs and NMS
    device = select_device('0' if torch.cuda.is_available() else 'cpu')  # default to GPU if available
    return hub_model.to(device)

model = custom(path_or_model='./yolov7/best.pt')  # custom example
#model = create(name='./yolov7/best.pt', pretrained=True, channels=3, classes=80, autoshape=True)  # pretrained example


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
  2                -1  1      2112  models.common.Conv                      [64, 32, 1, 1, None, 1, LeakyReLU(negative_slope=0.1)]
  3                -2  1      2112  models.common.Conv                      [64, 32, 1, 1, None, 1, LeakyReLU(negative_slope=0.1)]
  4                -1  1      9280  models.common.Conv                      [32, 32, 3, 1, None, 1, LeakyReLU(negative_slope=0.1)]
  5                -1  1      9280  models.common.Conv                      [32, 32, 3, 1, None, 1, LeakyReLU(negative_slope=0.1)]
  6  [-1, -2, -3, -4]  1         0  models.common.Concat                    [1]                           


###########


 76                73  1   1180672  models.common.Conv                      [256, 512, 3, 1, None, 1, LeakyReLU(negative_slope=0.1)]
 77      [74, 75, 76]  1     38780  yolov7.models.yolo.IDetect              [9, [[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]], [128, 256, 512]]
Model Summary: 263 layers, 6036636 parameters, 6036636 gradients, 13.2 GFLOPS

YOLOR 🚀 d44efc7 torch 2.2.1+cu121 CPU



Adding autoShape... 


### Test on img

In [7]:
img =  cv2.imread("./yolov7/car.png")
results = model(img)  # batched inference
results.print()
results.save()

image 1/1: 961x1715 2 Auto Privados, 1 Bus, 5 Camioneta rurals, 2 Auto colectivos, 1 Articulado
Speed: 5.9ms pre-process, 209.1ms inference, 7.4ms NMS per image at shape (1, 3, 384, 640)
Saved image0.jpg to runs/hub/exp8


In [ ]:
mydic = {}
for id,name in enumerate(results.names):
    mydic[id] = name

mydic

### Tracking

In [17]:
from ultralytics import YOLO
model_2 = YOLO(f'./yolov8n.pt')
#results_2 = model_2("./yolov7/car.png")
#results_2[0].boxes.xyxy
#print(dir(results[0].probs))
#detect = sv.Detections.from_ultralytics(results_2[0])
#https://supervision.roboflow.com/detection/core/
#https://supervision.roboflow.com/trackers/
#https://www.youtube.com/watch?v=OS5qI9YBkfk
#https://roboflow.com/how-to-track/yolov8
#https://www.youtube.com/watch?v=OS5qI9YBkfk&t=1011s
#https://docs.ultralytics.com/reference/engine/results/#ultralytics.engine.results.Results
#https://www.youtube.com/watch?v=oDALtKbprHg
#https://supervision.roboflow.com/detection/core/

In [22]:
import supervision as sv
tracker = sv.ByteTrack()
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results = model(frame)
    #print(results.xyxy[0].numpy()[:,:4])
    #detections = sv.Detections.from_ultralytics(results_2)
    detections = sv.Detections(xyxy=results.xyxy[0].numpy()[:,:4],
                               confidence=results.xyxy[0].numpy()[:,4],
                               class_id=results.xyxy[0].numpy()[:,-1].astype(int))
    #print(detections)
    detections = tracker.update_with_detections(detections)
    
    labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]

    annotated_frame = bounding_box_annotator.annotate(
        scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels)
    return annotated_frame

In [24]:
sv.process_video(
    source_path="./yolov7/video1_test2.mp4",
    target_path="./yolov7/video1_test2(copy).mp4",
    callback=callback
)

In [2]:
from pynput.mouse import Button, Controller
import time

mouse = Controller()

while True:
    mouse.click(Button.left, 1)
    time.sleep(30)

KeyboardInterrupt: 